## ESG Simulations using RRI

In [426]:
# ESG Threshold
# THRESHOLD = 35
# ESG Percentile
PERCENTILE = 0.5
THRESHOLD = 40

# metrics looked at 
METRICS = ['esg_score', 'environment', 'social', 'government']

# meet_esg - companies meeting esg criteria and RRI not zero
# not_esg - companies not meeting esg criteria and RRI not zero
# not_esg_zero - not_esg + companies with rri of 0
OPTIONS = ['meet_esg', 'not_esg', 'not_esg_zero', 'benchmark']


def liquidate(port, price, x, d):
    if len(port[x]['holdings']):
        prices = pd.Series([price[isin][price[isin]['date'] <= d].iloc[-1]['close'] for isin in port[x]['holdings'].index])
        prices.index = port[x]['holdings'].index
        port[x]['account_value'] = sum(prices * port[x]['holdings'])
    port[x]['history'].append(port[x]['account_value']) 
    return port


def allocate_holdings(portfolios, d, x, sector, df, metric, above, threshold, weighting):
    if sector == 'others': sector = 'other'
    group = df.loc[d][(df.loc[d]['sector'] == sector)]
    if above == 'above':
        group = group[group['mc'] > group['mc'].median()]
    else:
        group = group[group['mc'] <= group['mc'].median()]
    if threshold == 'threshold':
        if x == 'meet_esg':
            group = group[(group[metric] < THRESHOLD) & (group[metric] > 0)]
        elif x == 'not_esg':
            group = group[group[metric] >= THRESHOLD]
        elif x == 'not_esg_zero':
            group = group[(group[metric] >= THRESHOLD) | (group[metric] == 0)]
    else:
        if x == 'meet_esg':
            group = group[(0 < group[metric]) & (group[metric] < group[metric].quantile())]
        elif x == 'not_esg':
            group = group[group[metric] >= group[metric].quantile()]
        elif x == 'not_esg_zero':
            group = group[(group[metric] >= group[metric].quantile()) | (group[metric] == 0)]
    if weighting == 'marketcap':
        if len(group) == 0:
            portfolios[x]['holdings'] = pd.Series([])
            return portfolios
        tmc = sum(group['mc'] * group['freefloat'])
        portfolios[x]['holdings'] = portfolios[x]['account_value'] * (group['mc'] * group['freefloat'] / group['price'])/tmc
    else:
        if len(group) == 0:
            portfolios[x]['holdings'] = pd.Series([])
            return portfolios
        portfolios[x]['holdings'] = portfolios[x]['account_value']/len(group)/group['price']
    return portfolios


def simulate(datelist, price, df, name, frame, cutoff='threshold', allocate='equal'):        
    # simulate for each metric (rri, e, s, g, cc)
    for p1 in ['financials/util', 'others']: 
        for p2 in ['above', 'below']:
            for metric in METRICS: 
                portfolios = {x: {'account_value': 100000, 'holdings': {}, 'history': []} for x in OPTIONS}
                for date in tqdm(datelist):
                    for x in OPTIONS:
                        portfolios = liquidate(portfolios, price, x, date)
                        portfolios = allocate_holdings(portfolios, date, x, p1, df, metric, p2, cutoff, allocate)
                        
                # sell everything at the end
                for x in OPTIONS:
                    portfolios = liquidate(portfolios, price, x, date)
                    portfolios[x]['history'] = portfolios[x]['history'][1:]
                    title = f'{name}/{metric}/{x}/{p1}/{p2}/{frame}/{allocate}/{cutoff}'
                    returns[frame][title] = portfolios[x]['history']
                print(f"{name}/{metric}/{p1}/{p2}/{frame}/{allocate}/{cutoff}") 
                    
                # Calculate Portfolio Returns
                def portfolioMetrics(history, frame):
                    if frame == 'Yearly':
                        DIV = 1
                    elif frame == 'Quarterly':
                        DIV = 4
                    elif frame == 'Monthly':
                        DIV = 12
                    elif frame == 'Weekly':
                        DIV = 52
                    annual_return = (history[-1]/history[0])**(1/(len(history)/DIV)) - 1
                    annual_std = np.array([(history[i] - history[i - 1])/history[i - 1] for i in range(1, len(history))]).std() * np.sqrt(DIV)
                    print(f"Average Return (Per Annum): {rd(((history[-1]/history[0])**(1/len(history)))**DIV)}")
                    print(f"Final Value: {rd(history[-1])}\n")
                
                # Display Metrics
                print("\n\033[4mESG Portfolio Metrics\033[0m")
                portfolioMetrics(portfolios['meet_esg']['history'], frame)
                print("\n\033[4mNon-ESG Portfolio Metrics\033[0m")
                portfolioMetrics(portfolios['not_esg']['history'], frame)
                print("\n\033[4mNon-ESG/0-RRI Portfolio Metrics\033[0m")
                portfolioMetrics(portfolios['not_esg_zero']['history'], frame)
                print("\n\033[4mBenchmark Portfolio Metrics\033[0m")
                portfolioMetrics(portfolios['benchmark']['history'], frame)
                fig = make_subplots()
                fig.add_trace(go.Scatter(x=datelist, y=portfolios['benchmark']['history'], mode='lines', name='benchmark'))
                fig.add_trace(go.Scatter(x=datelist, y=portfolios['meet_esg']['history'], mode='lines', name='meet_esg'))
                fig.add_trace(go.Scatter(x=datelist, y=portfolios['not_esg']['history'], mode='lines', name='not_esg'))
                fig.add_trace(go.Scatter(x=datelist, y=portfolios['not_esg_zero']['history'], mode='lines', name='not esg and zero'))
                fig.update_layout(
                    title=f'Portfolio Performance of ESG Threshold at 35 ({metric})',
                    xaxis_title='Date',
                    yaxis_title='Portfolio Value',
                    legend_title='Portfolios',
                    template='plotly_dark', # Set dark theme
                    xaxis=dict(tickangle=-45) 
                )
                fig.show()

In [ ]:
%%time
for dates, t in [(quarterly_dates_spx, 'Quarterly'), (monthly_dates_spx, 'Monthly'), (weekly_dates_spx, 'Weekly')]:
    for alloc in ['marketcap', 'equal']:
        for cuto in ['threshold', 'percentile']:
            simulate(dates, spx_price, spx_df, 'SPX/normal', t, cuto, alloc)

 67%|█████████████████████████████████████████████████████▊                          | 492/731 [01:34<00:47,  5.02it/s]

In [ ]:
for dates, t in [(quarterly_dates_euro, 'Quarterly'), (monthly_dates_euro, 'Monthly'), (weekly_dates_euro, 'Weekly')]:
    for alloc in ['marketcap', 'equal']:
        for cuto in ['threshold', 'percentile']:
            simulate(dates, euro_price, euro_df, 'EURO/normal', t, cuto, alloc)

## ESG Simulations using Trend RRI